In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../Data/Split_highway.csv')

In [3]:
df.head()

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,id,version,tags,osm_type,fid,id_2,left,top,right,bottom
0,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Macondray Lane,NaN,...,5004035,-1,NaN,way,9789,9789,-122.417536,37.799515,-122.416396,37.798612
1,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11603,11603,-122.400434,37.719094,-122.399294,37.718191
2,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11718,11718,-122.399294,37.719094,-122.398154,37.718191
3,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11719,11719,-122.399294,37.718191,-122.398154,37.717287
4,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,5071582,-1,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11720,11720,-122.399294,37.717287,-122.398154,37.716383


In [5]:
df.columns

Index(['bicycle', 'bridge', 'cycleway', 'footway', 'highway', 'lanes', 'lit',
       'maxspeed', 'name', 'oneway', 'ref', 'service', 'sidewalk', 'surface',
       'id', 'version', 'tags', 'osm_type', 'fid', 'id_2', 'left', 'top',
       'right', 'bottom'],
      dtype='object')

In [6]:
df = df.drop(['version', 'id_2'], axis=1)

In [7]:
df['tags'].value_counts()

{"crossing":"marked"}                                                                                                                                                                                                                                             2126
{"layer":"-1"}                                                                                                                                                                                                                                                     493
{"tiger:cfcc":"A41","tiger:county":"San Francisco, CA","tiger:name_type":"St"}                                                                                                                                                                                     442
{"crossing":"unmarked"}                                                                                                                                                                                            

In [8]:
groups = df.groupby('fid')

In [9]:
tile_value_count_df = df['fid'].value_counts().sort_values().to_frame().reset_index().rename(columns={'index': 'tile_id','fid':'count'})

In [10]:
tile_value_count_df[tile_value_count_df['count']==1]['tile_id']

0        2898
1       12182
2       14240
3       14355
4       14470
        ...  
1337     2712
1338    13681
1339    11478
1340     2714
1341     4383
Name: tile_id, Length: 1342, dtype: int64

In [11]:
temp_tile_count_one = df[df['fid'].isin(tile_value_count_df[tile_value_count_df['count']==1]['tile_id'])]

In [12]:
temp_tile_count_one

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,sidewalk,surface,id,tags,osm_type,fid,left,top,right,bottom
37,NaN,NaN,NaN,NaN,residential,2.0,NaN,NaN,12th Avenue,NaN,...,both,NaN,8914970,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4670,-122.468841,37.748913,-122.467701,37.748010
46,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,12th Avenue,NaN,...,both,asphalt,8914974,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4537,-122.469981,37.765178,-122.468841,37.764275
48,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,12th Avenue,NaN,...,both,asphalt,8914974,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,4539,-122.469981,37.763371,-122.468841,37.762467
73,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Arelius Walker Drive,NaN,...,NaN,NaN,8914996,"{""old_name"":""Fitch Street"",""tiger:cfcc"":""A41"",...",way,14348,-122.373072,37.732648,-122.371932,37.731745
91,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Aquavista Way,NaN,...,NaN,NaN,8915002,"{""tiger:cfcc"":""A41"",""tiger:county"":""San Franci...",way,6620,-122.449459,37.753431,-122.448319,37.752528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83040,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4522,-122.469981,37.778732,-122.468841,37.777829
83042,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4524,-122.469981,37.776925,-122.468841,37.776021
83045,NaN,NaN,NaN,NaN,residential,NaN,yes,NaN,11th Avenue,NaN,...,both,asphalt,1115795705,"{""cycleway:both"":""no"",""parking:lane:both"":""par...",way,4527,-122.469981,37.774214,-122.468841,37.773311
83167,NaN,NaN,NaN,NaN,residential,2.0,NaN,25 mph,45th Avenue,no,...,both,asphalt,1117521594,"{""cycleway:both"":""no"",""parking:lane:left"":""per...",way,844,-122.506465,37.776925,-122.505325,37.776021


In [13]:
temp_tile_count_one.columns
for col in temp_tile_count_one.columns:
    print(f"{col}: {temp_tile_count_one[col].isnull().values.all()}, \
    {temp_tile_count_one[col].isnull().mean() * 100}")


bicycle: False,     92.10134128166915
bridge: False,     99.2548435171386
cycleway: False,     95.38002980625932
footway: False,     99.55290611028316
highway: False,     0.0
lanes: False,     83.90461997019374
lit: False,     62.74217585692996
maxspeed: False,     87.77943368107303
name: False,     21.982116244411326
oneway: False,     87.10879284649776
ref: False,     99.47839046199702
service: False,     99.2548435171386
sidewalk: False,     50.52160953800298
surface: False,     51.41579731743666
id: False,     0.0
tags: False,     16.98956780923994
osm_type: False,     0.0
fid: False,     0.0
left: False,     0.0
top: False,     0.0
right: False,     0.0
bottom: False,     0.0


In [14]:
df

,bicycle,bridge,cycleway,footway,highway,lanes,lit,maxspeed,name,oneway,...,sidewalk,surface,id,tags,osm_type,fid,left,top,right,bottom
0,NaN,NaN,NaN,NaN,residential,NaN,NaN,NaN,Macondray Lane,NaN,...,NaN,NaN,5004035,NaN,way,9789,-122.417536,37.799515,-122.416396,37.798612
1,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,NaN,concrete,5071582,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11603,-122.400434,37.719094,-122.399294,37.718191
2,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,NaN,concrete,5071582,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11718,-122.399294,37.719094,-122.398154,37.718191
3,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,NaN,concrete,5071582,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11719,-122.399294,37.718191,-122.398154,37.717287
4,no,NaN,NaN,NaN,motorway,5.0,NaN,55 mph,James Lick Freeway,yes,...,NaN,concrete,5071582,"{""NHS"":""STRAHNET"",""hgv"":""designated"",""hgv:nati...",way,11720,-122.399294,37.717287,-122.398154,37.716383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83994,NaN,NaN,NaN,NaN,footway,NaN,NaN,NaN,NaN,NaN,...,NaN,paved,1144261224,NaN,way,11752,-122.398154,37.792286,-122.397014,37.791383
83995,NaN,NaN,NaN,NaN,footway,NaN,NaN,NaN,NaN,NaN,...,NaN,paved,1144261224,NaN,way,11867,-122.397014,37.792286,-122.395874,37.791383
83996,NaN,NaN,NaN,NaN,footway,NaN,NaN,NaN,NaN,NaN,...,NaN,paved,1144261225,NaN,way,11867,-122.397014,37.792286,-122.395874,37.791383
83997,NaN,NaN,NaN,NaN,footway,NaN,NaN,NaN,NaN,NaN,...,NaN,paved,1144261226,NaN,way,11752,-122.398154,37.792286,-122.397014,37.791383


In [23]:
df[df['fid']==10725]['bicycle'].value_counts()

no     2
yes    2
Name: bicycle, dtype: int64

In [21]:
df['fid'].value_counts()

10725    63
11984    62
10721    62
7747     61
4340     60
         ..
3706      1
12440     1
9881      1
3254      1
2898      1
Name: fid, Length: 11795, dtype: int64

In [29]:
for col in df.columns:
    print(col)
    print(df[col].value_counts(dropna=False,normalize=True))
    print("\n\n\n")
# df['fid'].value_counts()

bicycle
NaN            0.924797
no             0.029215
yes            0.027965
designated     0.014072
dismount       0.002131
permissive     0.001238
private        0.000512
discouraged    0.000071
Name: bicycle, dtype: float64




bridge
NaN          0.982571
yes          0.016857
movable      0.000345
boardwalk    0.000202
gangway      0.000024
Name: bridge, dtype: float64




cycleway
NaN            0.962488
shared_lane    0.019750
lane           0.011453
track          0.003071
no             0.002357
separate       0.000512
crossing       0.000369
Name: cycleway, dtype: float64




footway
NaN               0.780259
sidewalk          0.135418
crossing          0.083870
ramp              0.000321
access_aisle      0.000060
square            0.000048
traffic_island    0.000024
Name: footway, dtype: float64




highway
footway           0.352564
residential       0.217669
service           0.158692
secondary         0.055560
tertiary          0.054155
primary           0.039512
ste

## ROAD VECTORS TILED 

In [3]:
df = pd.read_csv('../Data/Road_Vectors_Tiled.csv')

In [4]:
df.head()

,wkt_geom,fid,LINEARID,FULLNAME,RTTYP,MTFCC,id,left,top,right,bottom
0,MultiLineString ((-122.40410400000000379 37.74...,1,1.104990e+11,Hwy 101 S Off Rmp,M,S1400,11110,-122.404995,37.748913,-122.403855,37.748010
1,MultiLineString ((-122.47440799999999683 37.80...,2,1.104990e+11,Hwy 101 N on Rmp,M,S1400,4030,-122.474542,37.807648,-122.473402,37.806744
2,MultiLineString ((-122.47454194203001521 37.80...,3,1.104990e+11,Hwy 101 N on Rmp,M,S1400,3915,-122.475682,37.807648,-122.474542,37.806744
3,MultiLineString ((-122.41981637681401196 37.77...,4,1.108480e+12,US Hwy 101 Bus,U,S1200,9583,-122.419816,37.777829,-122.418676,37.776925
4,MultiLineString ((-122.41972054165864847 37.77...,5,1.108480e+12,US Hwy 101 Bus,U,S1200,9582,-122.419816,37.778732,-122.418676,37.777829


In [6]:
df.id.value_counts()

11109    25
10779    18
10777    17
8138     16
10994    16
         ..
10802     1
2275      1
2274      1
2272      1
7293      1
Name: id, Length: 10508, dtype: int64

In [16]:
df.RTTYP.value_counts(dropna=False)

M      25271
NaN     1193
I        423
S        357
U        286
O         11
Name: RTTYP, dtype: int64

In [15]:
df.MTFCC.value_counts()

S1400    24807
S1100      969
S1200      804
S1630      759
S1730       53
S1750       38
S1710       35
S1740       33
S1780       14
S1640       13
S1500        8
S1820        6
S1720        2
Name: MTFCC, dtype: int64

In [17]:
df_orig = pd.read_csv('../Data/Tiles_expanded.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 27, saw 367
